In [15]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn.model_selection import cross_val_score

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


# Bag of Words

In [5]:
# Utility function to create a list of the 200 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(200)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,cook,certainly,cat,consider,country,if,good,for,mad,musgrove,...,cry,make,believe,rabbit,day,bring,mrs,know,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,1,0,0,0,0,0,0,...,0,1,0,1,1,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


# Trying out Bag of Words

### Random Forest

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9642633228840125

Test set score: 0.8627819548872181


### Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 309) (3190,)
Training set score: 0.9200626959247649

Test set score: 0.9041353383458647


### Gradient Boosting

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8843260188087775

Test set score: 0.875


# Challenge 0

### Support Vector Machines

In [10]:
from sklearn import svm

clf = svm.SVC()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.803448275862069

Test set score: 0.8031015037593985


In [17]:
# Cross validation test
clf_test = cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1)
print('Cross Validation Accuracy Scores - Testing Set: {:.5f}(+/- {:.2f})'.format(clf_test.mean(), clf_test.std()*2))

Cross Validation Accuracy Scores - Testing Set: 0.78683(+/- 0.01)


In [18]:
# Cross validation test for logis
lr_test = cross_val_score(lr, X_train, y_train, cv=5, n_jobs=-1)
print('Cross Validation Accuracy Scores - Testing Set: {:.5f}(+/- {:.2f})'.format(lr_test.mean(), lr_test.std()*2))

Cross Validation Accuracy Scores - Testing Set: 0.89279(+/- 0.03)


Well, that wasn't really any better than of our other methods. It seems as though nothing seems to beat logistic regression for this dataset.

# Challenge 2:

Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below

In [11]:
# Print out all of the other available texts in gutenberg
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [12]:
# Fit and test other works of literature
def textcleaner(text):
    text = gutenberg.raw('shakespeare-macbeth.txt')
    text = re.sub(r'VOLUME \w+', '', text)
    text = re.sub(r'CHAPTER \w+', '', text)
    text = text_cleaner(text)
    print(text[:100])
    
    # Parse our cleaned data.
    text_doc = nlp(text)
    
    # Group into sentences.
    persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
    text_sents = [[sent, "Austen"] for sent in text_doc.sents]

    # Text is quite long, let's cut it down to the same length as Alice.
    text_sents = text_sents[0:len(alice_sents)]
    
    text_sentences = pd.DataFrame(text_sents)
    text_bow = bow_features(text_sentences, common_words)

    print('done')
    
    # Combine the text sentence data with the Alice data from the test set.
    X_text_test = np.concatenate((
        X_train[y_train[y_train=='Carroll'].index],
        text_bow.drop(['text_sentence','text_source'], 1)
    ), axis=0)
    y_text_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * text_bow.shape[0])])

    # Model.
    print('\nTest set score:', lr.score(X_text_test, y_text_test))
    lr_text_predicted = lr.predict(X_text_test)
    pd.crosstab(y_text_test, lr_text_predicted)

In [13]:
# Input and clean first 100 characters in macbeth
textcleaner('shakespeare-macbeth.txt')

Actus Primus. Scoena Prima. Thunder and Lightning. Enter three Witches. 1. When shall we three meet 
Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done

Test set score: 0.6133482475764355


Well, that didn't go very well, but it's not like it's all that surprising. Works of literature by different authors rarely are composed the same way, especially if they were not written in the same era, or the same themes. Still, 